In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install barbar

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import random
import numpy as np
import os

from barbar import Bar
%matplotlib inline

In [ ]:
EPOCH = 200
C = 100
BATCH = 1024
PATH = '/content/drive/My Drive/visual_media/cutmix'

writer = SummaryWriter(log_dir=PATH)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 2020
seed_everything(SEED)

In [ ]:
def cutmix(batch):
    input, target = batch

    ### Shuffle Minibatch ###
    indices = torch.randperm(input.size(0))
    input_s, target_s = input[indices], target[indices]

    lamb = np.random.uniform(0.0, 1.0)

    H, W = input.shape[2:]
    r_x = np.random.uniform(0, W)
    r_y = np.random.uniform(0, H)
    r_w = W * np.sqrt(1 - lamb)
    r_h = H * np.sqrt(1 - lamb)
    x1 = int(np.round(max(r_x - r_w / 2, 0)))
    x2 = int(np.round(min(r_x + r_w / 2, W)))
    y1 = int(np.round(max(r_y - r_h / 2, 0)))
    y2 = int(np.round(min(r_y + r_h / 2, H)))

    input[:, :, x1:x2, y1:y2] = input_s[:, :, x1:x2, y1:y2]
    target = (target, target_s, lamb)

    return input, target

In [ ]:
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH,
                                          shuffle=True, num_workers=2)

test_transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False).cuda()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [ ]:
for epoch in range(EPOCH):
    
    ################
    ### Training ###
    ################
    running_loss = 0.0
    for i, data in enumerate(Bar(trainloader)):
        data = cutmix(data)
        inputs, labels = data[0].cuda(), data[1]

        optimizer.zero_grad()

        outputs = net(inputs)

        target, target_s, lamb = labels[0].cuda(), labels[1].cuda(), labels[2]
        loss = lamb * criterion(outputs, target) + (1 - lamb) * criterion(outputs, target_s)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('[%d] loss: %.3f' %
          (epoch + 1, running_loss / 50000 * i))
    writer.add_scalar("loss", running_loss / 50000 * i, epoch)
    running_loss = 0.0


    ################
    ### Testing  ###
    ################
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].cuda(), data[1].cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
    writer.add_scalar("test_accuracy", 100 * correct / total, epoch)

    torch.save(net.state_dict(), PATH + '/model_latest.h5')
    scheduler.step()

50000/50000: [===============================>] - ETA 2.7s
[1] loss: 0.242
Accuracy of the network on the 10000 test images: 10 %
50000/50000: [===============================>] - ETA 0.6s
[2] loss: 0.203
Accuracy of the network on the 10000 test images: 14 %
50000/50000: [===============================>] - ETA 0.6s
[3] loss: 0.197
Accuracy of the network on the 10000 test images: 17 %
50000/50000: [===============================>] - ETA 0.6s
[4] loss: 0.191
Accuracy of the network on the 10000 test images: 19 %
50000/50000: [===============================>] - ETA 0.6s
[5] loss: 0.190
Accuracy of the network on the 10000 test images: 21 %
50000/50000: [===============================>] - ETA 0.6s
[6] loss: 0.186
Accuracy of the network on the 10000 test images: 22 %
50000/50000: [===============================>] - ETA 0.6s
[7] loss: 0.186
Accuracy of the network on the 10000 test images: 24 %
50000/50000: [===============================>] - ETA 0.6s
[8] loss: 0.181
Accuracy of the

In [ ]:
torch.save(net.state_dict(), PATH + '/model.h5')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.3f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 47.690 %
